# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from typing import Union
import random
from pathlib import Path

## Import Models and Data Loaders

In [4]:
from Return_dataloader import pass_dataloader
from Transformer_model import TabTransformer
from Plot_Accuracies import plot_loss_curves
from engine import train

# Device Agnostics

In [5]:
# Clear Cache
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Available device is: {device}")

Available device is: cpu


### Data Loader

In [6]:
train_dataloader, test_dataloader = pass_dataloader()

### Parameter Tune Function

In [7]:
from copy import deepcopy

def hyperparameter_tuning(param_grid,
                          train_dataloader,
                          test_dataloader,
                          device,
                          epochs=50):

    best_val_score = -float('inf')
    best_params = None
    best_results = None

    for params in param_grid:
        print(f"Training with d_model={params['d_model']}, nhead={params['nhead']}, "
              f"num_layers={params['num_layers']}, dropout={params['dropout']}, lr={params['lr']}")

        # Create model with given hyperparameters
        model = TabTransformer(input_dim=100,
                               d_model=params['d_model'],
                               nhead=params['nhead'],
                               num_layers=params['num_layers'],
                               dropout=params['dropout']).to(device)

        loss_fn = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=params['lr'],
                                     weight_decay=1e-4)

        # Train model and get results dictionary
        results = train(model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        device=device,
                        epochs=epochs)

        # Pick best validation score (ROC-AUC here) from all epochs
        max_val_roc_auc = max(results['test_roc_auc'])

        print(f"Max val ROC-AUC: {max_val_roc_auc:.4f}")

        if max_val_roc_auc > best_val_score:
            best_val_score = max_val_roc_auc
            best_params = params
            best_results = deepcopy(results)

    print(f"\nBest params: {best_params}")
    print(f"Best validation ROC-AUC: {best_val_score:.4f}")

    return best_params, best_results


## Hyperparameter Tuning

In [ ]:
from itertools import product
NUM_EPOCHS = 10


param_grid = {
    "d_model": [64, 128, 25],
    "nhead": [2, 4, 8, 16],
    "num_layers": [i * 2 for i in range(1,51)],
    "dropout": [0.0, 0.1 , 0.3 , 0.5],
    "lr": [0.1, 0.01, 0.001],
    "weight_decay": [0, 1e-4],
    "batch_size": [4, 8, 16, 32, 64]
}

param_grid = [
    dict(zip(param_grid.keys(), values))
    for values in product(*param_grid.values())
]
best_params, best_results = hyperparameter_tuning(
    param_grid,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    device=device,
    epochs=NUM_EPOCHS
)

Training with d_model=64, nhead=2, num_layers=2, dropout=0.0, lr=0.1


G:\M.Sc_Neaz\08_TCGA_Prostate_Data\Prostate_Cancer_Challenge\Python Scripts\pcvenv\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 10 | Train Loss: 0.1590 | Acc: 0.9502 | Prec: 0.9652 | Recall: 0.9259 | F1: 0.9451 | ROC-AUC: 0.9762 || Test Loss: 0.3372 | Acc: 0.8884 | Prec: 0.9835 | Recall: 0.8947 | F1: 0.9370 | ROC-AUC: 0.9048
Max val ROC-AUC: 0.9496
Training with d_model=64, nhead=2, num_layers=2, dropout=0.0, lr=0.1


  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 10 | Train Loss: 0.1289 | Acc: 0.9586 | Prec: 0.9816 | Recall: 0.9281 | F1: 0.9541 | ROC-AUC: 0.9860 || Test Loss: 0.2633 | Acc: 0.8837 | Prec: 0.9915 | Recall: 0.8822 | F1: 0.9337 | ROC-AUC: 0.9449
Max val ROC-AUC: 0.9629
Training with d_model=64, nhead=2, num_layers=6, dropout=0.1, lr=0.001


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 10 | Train Loss: 0.0328 | Acc: 0.9906 | Prec: 0.9898 | Recall: 0.9898 | F1: 0.9898 | ROC-AUC: 0.9986 || Test Loss: 0.4336 | Acc: 0.9279 | Prec: 0.9767 | Recall: 0.9449 | F1: 0.9605 | ROC-AUC: 0.9007
Max val ROC-AUC: 0.9474
Training with d_model=64, nhead=2, num_layers=6, dropout=0.1, lr=0.001


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# Attention Weights
import torch
import torch.nn as nn

class CustomTransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=256, dropout=0.5):
        super(CustomTransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = nn.GELU()

    def forward(self, src):
        # Self-attention with weights
        attn_output, attn_weights = self.self_attn(src, src, src, need_weights=True)
        src = src + self.dropout1(attn_output)
        src = self.norm1(src)

        ff_output = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(ff_output)
        src = self.norm2(src)

        return src, attn_weights  # Return both output and attention
        

class TabTransformerWithAttention(nn.Module):
    def __init__(self, input_dim,
                 d_model=128,
                 nhead=8,
                 num_layers=4,
                 dim_feedforward=256,
                 dropout=0.5):
        super(TabTransformerWithAttention, self).__init__()

        self.embedding = nn.Linear(input_dim, d_model)
        self.norm = nn.LayerNorm(d_model)

        # Stack custom encoder layers
        self.encoder_layers = nn.ModuleList([
            CustomTransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])

        self.classifier = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, input_dim)
        x = self.embedding(x)
        x = self.norm(x)

        all_attn_weights = []
        for layer in self.encoder_layers:
            x, attn_weights = layer(x)
            all_attn_weights.append(attn_weights)  # store attention maps

        x = x.mean(dim=1)
        logits = self.classifier(x)
        return logits, all_attn_weights  # return both predictions and attention maps
